In [26]:
pip install load_index_from_storage

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement load_index_from_storage (from versions: none)
ERROR: No matching distribution found for load_index_from_storage


In [1]:
import os
from llama_index import VectorStoreIndex, SimpleDirectoryReader
with open('openai_api.txt') as f:
    openai_api=f.read()
# openai_api
os.environ["OPENAI_API_KEY"] = openai_api
#os.environ["OPENAI_API_KEY"]

In [2]:
####### Importing the folder with pdf #######

documents=SimpleDirectoryReader("data").load_data()

In [3]:
############# Converting document to Index  

index=VectorStoreIndex.from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/202 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/197 [00:00<?, ?it/s]

In [4]:
query_engine=index.as_query_engine()
query_engine

In [5]:
response=query_engine.query("How many types of conducting material are there?")

In [6]:
print(response)

There are two types of conducting materials mentioned in the context: copper and aluminium.


In [7]:
response2=query_engine.query("What is Maxwell's Loop Current Method?")
print(response2)#### Shows the best response

Maxwell's Loop Current Method, also known as Mesh Analysis, is a method used to determine branch currents and voltages across the elements of a network. In this method, loop currents are taken instead of branch currents, and these loop currents are assumed to flow in the clockwise direction. Branch currents can be found in terms of loop currents, and the sign conventions for the IR drops and battery emfs are the same as for Kirchhoff's law. This method is particularly useful when all the sources in the network are given as voltage sources, and if there is a current source present, it is converted into an equivalent voltage source.


In [8]:
from llama_index.response.pprint_utils import pprint_response
pprint_response(response2)

Final Response: Maxwell's Loop Current Method, also known as Mesh
Analysis, is a method used to determine branch currents and voltages
across the elements of a network. In this method, loop currents are
taken instead of branch currents, and these loop currents are assumed
to flow in the clockwise direction. Branch currents can be found in
terms of loop currents, and the sign conventions for the IR drops and
battery emfs are the same as for Kirchhoff's law. This method is
particularly useful when all the sources in the network are given as
voltage sources, and if there is a current source present, it is
converted into an equivalent voltage source.


In [9]:
pprint_response(response2, show_source=True)### Shows all the diiferent souces and output
###### Final Response is the best possible response

Final Response: Maxwell's Loop Current Method, also known as Mesh
Analysis, is a method used to determine branch currents and voltages
across the elements of a network. In this method, loop currents are
taken instead of branch currents, and these loop currents are assumed
to flow in the clockwise direction. Branch currents can be found in
terms of loop currents, and the sign conventions for the IR drops and
battery emfs are the same as for Kirchhoff's law. This method is
particularly useful when all the sources in the network are given as
voltage sources, and if there is a current source present, it is
converted into an equivalent voltage source.
______________________________________________________________________
Source Node 1/2
Node ID: 0ec046d5-0074-4757-a59e-87603ce2c94a
Similarity: 0.8774012413681515
Text: Example :- Write the loop equation for the given circuit be low
(Supplementary exam 2004 )  ir1E1 r2 E2 r3E3  Solution:  Apply KVL to
the loop,  ( )1 1 2 2 3 3  1 2 3 1 2 3  1

# Finding Top K responses

In [12]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor

In [18]:
retriever=VectorIndexRetriever(index=index,similarity_top_k=3)

query_engine=RetrieverQueryEngine(retriever=retriever)

In [19]:
response_1=query_engine.query("How many types of conducting material are there?")
print(response_1)

There are two types of conducting materials mentioned in the context: copper and aluminium.


In [21]:
response2=query_engine.query("What is Maxwell's Loop Current Method?")
response2

Response(response="Maxwell's Loop Current Method is a technique used to determine branch currents and voltages across the elements of a network. In this method, loop currents are taken instead of branch currents, and they are assumed to flow in the clockwise direction. Branch currents can be found in terms of loop currents. The sign conventions for the IR drops and battery emfs are the same as for Kirchhoff's law. This method is easier to apply when all the sources in the network are given as voltage sources. If there is a current source present, it is converted into an equivalent voltage source.", source_nodes=[NodeWithScore(node=TextNode(id_='0ec046d5-0074-4757-a59e-87603ce2c94a', embedding=None, metadata={'page_label': '4', 'file_name': 'lecture.pdf', 'file_path': 'data\\lecture.pdf', 'file_type': 'application/pdf', 'file_size': 8825573, 'creation_date': '2024-01-30', 'last_modified_date': '2024-01-30', 'last_accessed_date': '2024-01-30'}, excluded_embed_metadata_keys=['file_name', 

# Similarity Post Processor

In [25]:
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.8)
#, node_postprocessor=postprocessor
query_engine=RetrieverQueryEngine(retriever=retriever, node_postprocessors=[postprocessor])
response2=query_engine.query("What is Maxwell's Loop Current Method?")
print(response2)

Maxwell's Loop Current Method is a technique used to determine branch currents and voltages across the elements of a network. In this method, loop currents are taken instead of branch currents, and they are assumed to flow in the clockwise direction. Branch currents can be found in terms of loop currents, and the sign conventions for the IR drops and battery emfs are the same as for Kirchhoff's law. This method is particularly useful when all the sources in the network are given as voltage sources, and if there is a current source present, it can be converted into an equivalent voltage source.


# Creating Storage in Hard Disk

In [28]:
from llama_index import load_index_from_storage

In [29]:
PERSIT_DIR="./storage"
if not os.path.exists(PERSIT_DIR):
    #####load the documents and create the index
    documents=SimpleDirectoryReader("data").load_data()
    index=VectorStoreIndex.from_documents(documents)
    ######### store it for later
    index.storage_context.persist(persist_dir=PERSIT_DIR)
else:
    ####load the existing index
    storage_context=StorageContext.from_defaults(persist_dir=PERSIT_DIR)
    index=load_index_from_storage(storage_context)
query_engine=index.as_query_engine()
response=query_engine.query("What is Maxwell's Loop Current Method?")
print(response)

Maxwell's Loop Current Method, also known as Mesh Analysis, is a method used to determine branch currents and voltages across the elements of a network. In this method, loop currents are used instead of branch currents, and these loop currents are assumed to flow in the clockwise direction. By applying Kirchhoff's Voltage Law (KVL) to each loop, the branch currents can be found in terms of the loop currents. The sign conventions for the voltage drops and battery electromotive forces (emfs) are the same as in Kirchhoff's law. This method is particularly useful when all the sources in the network are given as voltage sources, and if there is a current source present, it can be converted into an equivalent voltage source.
